In [2]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config
%%bash 
pwd

In [11]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [58]:
from core.dl_framework.learner import Learner
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_trainset_ds
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
# train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(valid_ds, bs)

In [66]:
learn = Learner(x_train, setup_config)
vars(learn)
# import matplotlib.pyplot as plt
# plt.imshow(learn.data.train_ds[0][0].float().reshape(28, -1))
#train_ds, valid_s = split_trainset_ds(train_ds, 0.2)
#len(train_ds)

{'model': 'Model_2',
 'opt': 'adam',
 'loss': 'cross_entropy',
 'bs': 64,
 'data': <core.dl_framework.learner.Learner at 0x7fd88d892128>,
 'cbh': <core.dl_framework.callbacks.CallbackHandler at 0x7fd88d892470>,
 'recorder': core.dl_framework.callbacks.Recorder_Cb}

In [57]:
from core.dl_framework.data import split_trainset_ds
split_trainset_ds()


not hi
